# Implementierung eines einfachen Tokenizers

Ziel dieser Übung ist es, diverse Texte einzulesen, diese zu tokenisieren und als Folge von Tokens auszugeben. Hierbei liegt der Fokus nicht auf Vollständigkeit! Die Eingabetexte müssen nicht *perfekt* tokenisiert werden, nur so gut wie möglich verarbeitet werden.

## 1. Einlesen und Vorverarbeitung der Texte

Im Ordner `tokenizer_data` befinden sich mehrere Dateien mit Eingabetexten in drei verschiedenen Eingabeformaten. Die ersten Schritte sind nun, diese Dateien einzulesen, die Texte zu extrahieren und von unerwünschten Elementen zu säubern, sowie die reinen Texte gesammelt in eine dafür geeignete Datenstruktur zu legen.

`cmc_train_whats_app.txt` und `cmc_train_wiki_discussion_2.txt` sind einfache Textdateien, die durch Leerzeilen getrennte Einzeltexte enthalten. `cmc_train_whats_app.txt` enthält einzelne WhatsApp-Nachrichten. `cmc_train_wiki_discussion_2.txt` enthält einzelne Beiträge von Wikipedia-Diskussionsseiten. Jeder Text hat dabei eine Metadatenkopfzeile in einem XML-ähnlichen Format. Diese Kopfzeile soll entfernt werden, und die einzelnen Texte sollen getrennt bereitgestellt werden.

`epub_alice1.xml` und `epub_mittelpunkt3.xml` enthalten jeweils ein einzelnes Kapitel aus einer E-Book-Ausgabe von Lewis Carrolls "Alice's Adventures in Wonderland" und Jules Vernes "Reise nach dem Mittelpunkt der Erde". Die Kapiteldateien selbst liegen als XHTML-Dokumente vor und stammen so unverändert aus den ursprünglichen Epub-Dateien, aus denen sie extrahiert wurden. Bei diesen Dateien sollen 
1. alle Absätze des eigentlichen Textes extrahiert werden,
2. der Text der Absätze von allen XML-Tags und Zeilenumbrüchen befreit werden,
3. die einzelnen Absätze durch Zeilenumbrüche getrennt zusammengefügt werden.

`twitter-corpus.csv` enthält Tweets als CSV-Tabelle. Hier sollen aus der Spalte "TweetText" die einzelnen Tweets als Texte bereitgestellt werden. Zum Einlesen der CSV-Datei kann entweder wieder Pandas mit der Funktion `read_table()` oder das `csv`-Modul aus der Python-Standardbibliothek verwendet werden (siehe Übung 3.1).

Es bietet sich an, die einzelnen Texte in einer Liste oder einem Dictionary abzuspeichern.

In [1]:
# Imports
import re

'''Liste an Dateinamen der einzulesenden Dateien'''
filenames = [
    'tokenizer_data/cmc_train_whats_app.txt',
    'tokenizer_data/cmc_train_wiki_discussion_2.txt',
    'tokenizer_data/epub_alice1.xml',
    'tokenizer_data/epub_mittelpunkt3.xml',
    'tokenizer_data/twitter-corpus.csv', 
]

In [2]:
'''Diese Funktion liest die Texte aus den cmc_*.txt-Dateien ein und gibt sie als Liste zurück'''
def extract_txt(filename):
    with open(filename, mode='r', encoding='utf-8-sig') as infile:
        content = infile.read()
    texts = re.split(r'\n+', content)
    texts = [text.strip() for text in texts if text and not re.match(r'<.+?>', text)]
    
    return texts

In [3]:
'''Diese Funktion liest die Texte aus den epub_*.xml-Dateien ein und gibt sie als Liste zurück'''
def extract_epub(filename): 
    with open(filename, mode='r', encoding='utf-8-sig') as infile:
        content = infile.read()
    content = re.sub('\n', ' ', content)
    paragraphs = re.findall(r'<p>(.+?)</p>', content)
    text = ' '.join(paragraphs)
    text = re.sub(r'<.+?>', '', text)
    
    return [text]

In [4]:
import csv

'''Diese Funktion liest die Texte aus der twitter-corpus.csv-Datei ein und gibt sie als Liste zurück'''
def extract_twitter(filename):
    texts = []
    
    with open(filename, mode='r', encoding='utf-8-sig') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            texts.append(row['TweetText'])
    
    return texts

In [5]:
### Hier Code vervollständigen

# Hier sollen die Dateien eingelesen und die einzelnen Texte in eine geeignete Datenstruktur abgelegt werden
texts = {}

for filename in filenames:
    extracted = []
    if filename.endswith('.xml'):
        extracted = extract_epub(filename)
    elif filename.endswith('.csv'):
        extracted = extract_twitter(filename)
    elif filename.endswith('.txt'):
        extracted = extract_txt(filename)
    
    if extracted:
        texts[filename.split('/')[1]] = extracted
        
texts.keys()

dict_keys(['cmc_train_whats_app.txt', 'cmc_train_wiki_discussion_2.txt', 'epub_alice1.xml', 'epub_mittelpunkt3.xml', 'twitter-corpus.csv'])

## 2. Tokenisierung der Texte

Die im vorherigen Schritt gewonnenen Texte sollen nun tokenisiert werden. Dazu müssen anhand des untenstehenden Code-Gerüsts folgende  Teilaufgaben gelöst werden:

1. Die Funktion `tokenize()` vervollständigen. Die grobe Funktionsweise der Funktion ist im Quellcode näher beschrieben.
2. Die regulären Ausdrücke bzw. Token-Klassen zum Tokenisieren vervollständigen.

Eine Token-Klasse bezieht sich auf eine Art von Eingabe, die gesondert behandelt werden muss/kann. Typische Beispiele für gesonderte Token-Klassen sind bspw. Zahlen, URLs und "normale" Wörter. 
Die Token-Klassen und regulären Ausdrücke liegen dabei in einer Hierarchie, da sie der Reihe nach durchlaufen werden. Es ist wichtig, dass dabei die speziellen Token-Klassen in der Hierarchie über den generischen stehen, sprich als erstes angegeben werden. Dadurch, dass in jedem Tokenisierungsschritt die gefundenen Tokens aus dem Eingabetext "konsumiert" werden, könnten sonst generischere Token-Klassen den speziellen den Input "wegnehmen".
Verdeutlicht am Beispieltext und den Token-Klassen aus der Angabe:
```python
token_classes = {
    'emoasc': r'[:;=][-^]?[DP()c|\[\]{}]|XD+',
    'punct': r'[,.:;()]',
    'word': r'\w+'
}
```
Wenn nun der Eingabetext `Das ist ein Test. :)` tokenisiert wird, wird in der gegebenen Hierarchie als erstes nach Smileys gesucht, dann nach Satzzeichen, dann nach allen anderen zusammenhängenden Wörtern. In diesem Fall stellt "emoasc" eine speziellere Klasse für "gewisse Kombinationen von Satzzeichen" dar, während "punct" eine allgemeine Klasse für "alle Satzzeichen" darstellt. Würde man die Klassen "emoasc" und "punct" vertauschen, könnten ":" und ")" als einzelne Satzzeichen im Verarbeitungsschritt von "punct" interpretiert werden und nie das Erkennungsmuster für "emoasc" erreichen!

Welche besonderen und interessanten Tokenklassen gibt es in den Eingabetexten? Gibt es Arten von Tokens, die spezifisch für die Art des Eingabetextes sind? 

In [6]:
class Token():
    '''Ein Objekt, das ein Token repräsentiert.
    Jedes Token hat einen `text` und eine `token_class`, welche die Klasse (also Art) des Tokens angibt. Außerdem werden
    seine Start- und Endposition im ursprünglichen Text gespeichert'''
    
    def __init__(self, text, token_class, start, end):
        self.text = text
        self.token_class = token_class
        self.start = start
        self.end = end
        
    def __str__(self):
        return self.text
    
    def __repr__(self):
        return f"'{self.text}'"

def debug(text, n = 0):
    '''Diese Funktion tokenisiert den gegebenen Text mit `tokenize()` und gibt zu jedem Token alle Informationen aus'''
    tokens = tokenize(text)
    for t in tokens[:n]:
        print(f'"{t.text}":\tclass="{t.token_class}", start="{t.start}", end="{t.end}"')

In [7]:
### Hier Code zu 1. vervollständigen ###

def tokenize(text):
    '''Diese Funktion tokenisiert den in `text` gegebenen Text und gibt eine Liste an Tokens zurück.
    
    Zum Tokenisieren des Eingabetextes müssen die regulären Ausdrücke in `token_classes` der Reihe nach auf den Eingabetext
    angewendet werden. Jedes gefundene Token soll als Token-Objekt in die `tokens`-Liste aufgenommen werden.
    Vor dem Anwenden des nächsten regulären Ausdrucks müssen außerdem alle in diesem Schritt gefundenen Tokens aus dem
    Eingabetext entfernt werden.
    Nach Durchlaufen der kompletten Tokenisierungshierarchie sollen die Tokens als Liste zurückgegeben werden. 
    
    Achtung: Achte darauf, dass die Tokens in `tokens` vor der Rückgabe in der richtigen Reihenfolge stehen! 
    '''
    
    tokens = []
    
    for token_type, regex in token_classes.items():
        for match in re.finditer(regex, text):
            tokens.append(Token(match[0], token_type, match.start(), match.end()))
            text = re.sub(regex, ' ' * len(match[0]), text, count=1)
    
    # Die Tokens sind noch nicht in der richtigen Reihenfolge und müssen erst nach ihrer Startposition sortiert werden
    return sorted(tokens, key = lambda token: token.start)

In [8]:
### Hier Code zu 2. vervollständigen ###

'''Die einzelnen regulären Ausdrücke zum Tokenisieren der einzelnen Token-Klassen.
Die Schlüssel des Dictionaries sind die Namen der Token-Klassen,
die Werte sind die zugehörigen regulären Ausdrücke.
Beim Tokenisieren werden die Tokenizer der Reihe nach aufgerufen, beginnend mit dem ersten Element im Dictionary.'''
token_classes = {
    'url': r"(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", # https://gist.github.com/gruber/249502
    'email': r'(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)', # https://emailregex.com/
    'emoasc': r'[:;=][-^]?[DP()c|\[\]{}]|[xX]D+|\*_\*',
    'acronym': r'([A-ZÄÖÜ]\.){2,}',
    'emoji': r'emojiQ[a-z]+(:?[A-Z][a-z]+)*',
    'hashtag': r'#[\w-]+',
    'twitter_handle': r'@[\w-]+',
    'ordnum': r'[0-9]+(st|nd|rd|th)',
    'word': r"[\w']+(-[\w']+)*",
    'cardnum': r'[0-9]+([,.][0-9]+)?(st|nd|rd|th)?',
    'punct': r'\.\.\.|[,.:;()?!-"„“”]',
    'sym': r'\S'
}

In [9]:
key = 'twitter-corpus.csv'
print('\n================', key, '================\n')
for text in texts[key][300:320]:
    print(text, '\n')
    debug(text, 100)
    print('\n')


================ twitter-corpus.csv ================

Every time I try the voice control on my iPod Touch to send an iMessage, it starts playing "Who Knew" by Pink.  Still not Flawless @Apple 

"Every":	class="word", start="0", end="5"
"time":	class="word", start="6", end="10"
"I":	class="word", start="11", end="12"
"try":	class="word", start="13", end="16"
"the":	class="word", start="17", end="20"
"voice":	class="word", start="21", end="26"
"control":	class="word", start="27", end="34"
"on":	class="word", start="35", end="37"
"my":	class="word", start="38", end="40"
"iPod":	class="word", start="41", end="45"
"Touch":	class="word", start="46", end="51"
"to":	class="word", start="52", end="54"
"send":	class="word", start="55", end="59"
"an":	class="word", start="60", end="62"
"iMessage":	class="word", start="63", end="71"
",":	class="punct", start="71", end="72"
"it":	class="word", start="73", end="75"
"starts":	class="word", start="76", end="82"
"playing":	class="word", start="83", en

## 3. Ausgabe der Texte

Abschließend sollten alle Texte in tokenisierter Form als Datei ausgegeben werden. Ein gängiges Format dafür ist, die Texte als einfache Textdateien auszugeben, wobei immer ein Token pro Zeile steht. D. h.:
```
Ich
habe
alle
meine
Tokens
auf
separaten
Zeilen
!
=D
```
Dies kann einfach über die in Python eingebauten Dateioperationen geschehen, sprich z. B. über `file.write()`. Dabei sollte am besten für jede Eingabedatei eine Ausgabedatei entstehen.

In [10]:
# Hier sollen die tokenisierten Texte ausgegeben werden.
import os

os.makedirs('tokenizer_output', exist_ok=True)

for key in texts:
    with open(f'tokenizer_output/{key}.tkns', mode='w', encoding='utf-8') as outfile:
        for text in texts[key]:
            for token in tokenize(text):
                outfile.write(token.text + '\n')
            outfile.write('\n')